In [ ]:
import cv2
import os
import pandas as pd

def create_skeleton_video(original_video_path, csv_path, output_video_path, output_video_name='skeleton_overlay.mp4', scale_factor=1.0):
    """
    기존에 생성된 CSV 파일과 원본 영상을 이용하여 스켈레톤(18개 랜드마크)이 그려진 영상을 생성합니다.
    
    Args:
        original_video_path (str): 원본 영상의 절대 경로.
        csv_path (str): CSV 파일의 절대 경로 (스켈레톤 데이터가 기록됨).
        output_video_path (str): 결과 영상을 저장할 디렉토리.
        output_video_name (str): 생성될 영상 파일 이름 (기본: 'skeleton_overlay.mp4').
        scale_factor (float): CSV에 기록된 상대 좌표에 곱할 확대 계수 (기본: 1.0). 
                              (영상의 중앙을 기준으로 좌표를 재배치합니다.)
    """
    # 원본 영상 열기
    cap = cv2.VideoCapture(original_video_path)
    if not cap.isOpened():
        print("Error: Unable to open video file at path:", original_video_path)
        return

    # 영상 속성 읽기
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # CSV 파일 읽기 (각 행: frame, landmark, x, y, z, accuracy)
    df = pd.read_csv(csv_path)
    # 그룹핑: 각 프레임에 해당하는 행들을 빠르게 찾기 위해
    grouped = df.groupby('frame')

    # 출력 영상 저장 경로 확인
    if not os.path.exists(output_video_path):
        os.makedirs(output_video_path)
    output_video_file = os.path.join(output_video_path, output_video_name)
    out = cv2.VideoWriter(output_video_file, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
    
    # 영상 중앙 좌표 (절대 좌표)
    center_x = frame_width // 2
    center_y = frame_height // 2
    # scale: CSV에 기록된 상대 좌표에 곱할 값
    scale = frame_width * scale_factor

    # (선택적) 스켈레톤 연결선 정의 (부위 이름을 이용)
    connections = [
        ("Nose", "Neck"),
        ("Neck", "Right Shoulder"),
        ("Right Shoulder", "Right Elbow"),
        ("Right Elbow", "Right Wrist"),
        ("Neck", "Left Shoulder"),
        ("Left Shoulder", "Left Elbow"),
        ("Left Elbow", "Left Wrist"),
        ("Neck", "Right Hip"),
        ("Right Hip", "Right Knee"),
        ("Right Knee", "Right Ankle"),
        ("Neck", "Left Hip"),
        ("Left Hip", "Left Knee"),
        ("Left Knee", "Left Ankle"),
        ("Nose", "Right Eye"),
        ("Right Eye", "Right Ear"),
        ("Nose", "Left Eye"),
        ("Left Eye", "Left Ear")
    ]
    
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print("모든 프레임 처리 완료, 총 프레임 수:", frame_idx)
            break

        # CSV에 frame_idx에 해당하는 데이터가 있으면 처리
        if frame_idx in grouped.groups:
            rows = grouped.get_group(frame_idx)
            # 딕셔너리: key = 부위 이름, value = (abs_x, abs_y)
            landmarks_coords = {}
            for _, row in rows.iterrows():
                # CSV에 기록된 x, y는 상대 좌표이므로, 절대 좌표로 변환
                # (영상 중앙을 기준으로 scale_factor * (상대좌표)를 더함)
                abs_x = int(center_x + row['x'] * scale)
                abs_y = int(center_y + row['y'] * scale)
                landmarks_coords[row['landmark']] = (abs_x, abs_y)
                # 부위 이름과 원을 영상에 그리기
                cv2.circle(frame, (abs_x, abs_y), 5, (0, 255, 0), -1)
                cv2.putText(frame, row['landmark'], (abs_x, abs_y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            
            # 연결선 그리기
            for conn in connections:
                if conn[0] in landmarks_coords and conn[1] in landmarks_coords:
                    pt1 = landmarks_coords[conn[0]]
                    pt2 = landmarks_coords[conn[1]]
                    cv2.line(frame, pt1, pt2, (255, 0, 0), 2)
        
        # 출력 영상에 프레임 기록
        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()
    print("Skeleton overlay video saved at:", output_video_file)

# --- 실행 부분 ---
# 원본 영상의 절대 경로
original_video_path = '/Users/dina/Documents/GitHub/action_class/AI_Train/temp/test_vid.mp4'
# CSV 파일의 절대 경로 (이전에 생성된 CSV)
csv_path = '/Users/dina/Documents/GitHub/action_class/AI_Train/Dataset_csv/skeleton_video.csv'
# 결과 영상은 현재 작업 디렉토리(예: AI_Train/Dataset_csv)에 저장
output_video_path = os.getcwd()
create_skeleton_video(original_video_path, csv_path, output_video_path, output_video_name='skeleton_overlay.mp4', scale_factor=1.0)
